In [2]:
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org beautifulsoup4
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org halo


!pip install --index-url https://pypi.org/simple/ openai
!pip install --index-url https://pypi.org/simple/ pypdf2
!pip install --index-url https://pypi.org/simple/ pyyaml


Looking in indexes: https://mirrors.zju.edu.cn/pypi/simple
Could not fetch URL https://mirrors.zju.edu.cn/pypi/simple/beautifulsoup4/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='mirrors.zju.edu.cn', port=443): Max retries exceeded with url: /pypi/simple/beautifulsoup4/ (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1131)'))) - skipping
Could not fetch URL https://mirrors.zju.edu.cn/pypi/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='mirrors.zju.edu.cn', port=443): Max retries exceeded with url: /pypi/simple/pip/ (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1131)'))) - skipping


ERROR: Could not find a version that satisfies the requirement beautifulsoup4 (from versions: none)
ERROR: No matching distribution found for beautifulsoup4


Looking in indexes: https://mirrors.zju.edu.cn/pypi/simple
Could not fetch URL https://mirrors.zju.edu.cn/pypi/simple/halo/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='mirrors.zju.edu.cn', port=443): Max retries exceeded with url: /pypi/simple/halo/ (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1131)'))) - skipping
Could not fetch URL https://mirrors.zju.edu.cn/pypi/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='mirrors.zju.edu.cn', port=443): Max retries exceeded with url: /pypi/simple/pip/ (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1131)'))) - skipping


ERROR: Could not find a version that satisfies the requirement halo (from versions: none)
ERROR: No matching distribution found for halo


## Generate Reports

In [7]:
!pip install --index-url https://pypi.org/simple/ chardet


Looking in indexes: https://pypi.org/simple/
  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   ------ --------------------------------- 30.7/199.4 kB 77.0 kB/s eta 0:00:03
   ------ --------------------------------- 30.7/199.4 kB 77.0 kB/s eta 0:00:03
   -------- -----------

In [11]:
import openai
from time import time, sleep
from halo import Halo
import textwrap
import yaml

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [ ]:
###     file operations

def save_file(filepath, content):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as infile:
        return infile.read()

def save_yaml(filepath, data):
    with open(filepath, 'w', encoding='utf-8') as file:
        yaml.dump(data, file, allow_unicode=True)

def open_yaml(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data

In [ ]:
###     API functions

def chatbot(conversation, model="gpt-4-0613", temperature=0):
    max_retry = 7
    retry = 0
    while True:
        try:
            spinner = Halo(text='Thinking...', spinner='dots')
            spinner.start()
            
            response = openai.ChatCompletion.create(model=model, messages=conversation, temperature=temperature)
            text = response['choices'][0]['message']['content']

            spinner.stop()
            
            return text, response['usage']['total_tokens']
        except Exception as oops:
            print(f'\n\nError communicating with OpenAI: "{oops}"')
            if 'maximum context length' in str(oops):
                a = conversation.pop(0)
                print('\n\n DEBUG: Trimming oldest message')
                continue
            retry += 1
            if retry >= max_retry:
                print(f"\n\nExiting due to excessive errors in API: {oops}")
                exit(1)
            print(f'\n\nRetrying in {2 ** (retry - 1) * 5} seconds...')
            sleep(2 ** (retry - 1) * 5)

def chat_print(text):
    formatted_lines = [textwrap.fill(line, width=120, initial_indent='    ', subsequent_indent='    ') for line in text.split('\n')]
    formatted_text = '\n'.join(formatted_lines)
    print('\n\n\nCHATBOT:\n\n%s' % formatted_text)

In [ ]:
if __name__ == '__main__':
    openai.api_key = open_file('key_openai.txt').strip()
    paper = open_file('input.txt')
    if len(paper) > 22000:
        paper = paper[0:22000]
    ALL_MESSAGES = [{'role':'system', 'content': paper}]
    while True:
        # get user input
        text = input('\n\n\nUSER:\n\n')
        if text == '':
            # empty submission, probably on accident
            continue
        ALL_MESSAGES.append({'role': 'user', 'content': text})
        
        # get response
        response, tokens = chatbot(ALL_MESSAGES)
        if tokens >= 7800:
            a = ALL_MESSAGES.pop(1)
        chat_print(response)
        ALL_MESSAGES.append({'role': 'assistant', 'content': response})

## Chat

In [ ]:
import openai
from time import time, sleep
from halo import Halo
import textwrap
import yaml

In [ ]:
###     file operations

def save_file(filepath, content):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as infile:
        return infile.read()

def save_yaml(filepath, data):
    with open(filepath, 'w', encoding='utf-8') as file:
        yaml.dump(data, file, allow_unicode=True)

def open_yaml(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    return data

In [ ]:
###     API functions

def chatbot(conversation, model="gpt-4-0613", temperature=0):
    max_retry = 7
    retry = 0
    while True:
        try:
            spinner = Halo(text='Thinking...', spinner='dots')
            spinner.start()
            
            response = openai.ChatCompletion.create(model=model, messages=conversation, temperature=temperature)
            text = response['choices'][0]['message']['content']

            spinner.stop()
            
            return text, response['usage']['total_tokens']
        except Exception as oops:
            print(f'\n\nError communicating with OpenAI: "{oops}"')
            if 'maximum context length' in str(oops):
                a = conversation.pop(0)
                print('\n\n DEBUG: Trimming oldest message')
                continue
            retry += 1
            if retry >= max_retry:
                print(f"\n\nExiting due to excessive errors in API: {oops}")
                exit(1)
            print(f'\n\nRetrying in {2 ** (retry - 1) * 5} seconds...')
            sleep(2 ** (retry - 1) * 5)

def chat_print(text):
    formatted_lines = [textwrap.fill(line, width=120, initial_indent='    ', subsequent_indent='    ') for line in text.split('\n')]
    formatted_text = '\n'.join(formatted_lines)
    print('\n\n\nCHATBOT:\n\n%s' % formatted_text)

In [ ]:
if __name__ == '__main__':
    openai.api_key = open_file('key_openai.txt').strip()
    paper = open_file('input.txt')
    if len(paper) > 22000:
        paper = paper[0:22000]
    ALL_MESSAGES = [{'role':'system', 'content': paper}]
    while True:
        # get user input
        text = input('\n\n\nUSER:\n\n')
        if text == '':
            # empty submission, probably on accident
            continue
        ALL_MESSAGES.append({'role': 'user', 'content': text})
        
        # get response
        response, tokens = chatbot(ALL_MESSAGES)
        if tokens >= 7800:
            a = ALL_MESSAGES.pop(1)
        chat_print(response)
        ALL_MESSAGES.append({'role': 'assistant', 'content': response})